## **SISTEMA RECOMENDADOR DE NOTICIAS PARA CLIENTES CORPORATIVOS DE UN BANCO**


**Profesor**: Juan Fernando Perez Bernal

**Integrantes**: Miller Puentes, Joan Esteban Chacón, Grace González, Catalina Cárdenasace González, Catalina Cárdenas


En este documento se presenta el Modelamiento de los datos para el desarrollo de un sistema de recomendación de noticias para los clientes de un banco basado en el sector económico al que pertenece.

### ***Tabla de contenido***

**0.   Instalación y llamada de librerías**

  0.1 Instalación de pyLDAvis

  0.2 Instalación de librerías


**4. Recomendadores**

  4.1 Recomendador: LDA - Stemming

  4.2 Recomendador: LDA - Lematización

**5. Resultados**


## ***0. Instalación y llamada de librerías***

### ***0.1 Instalación de pyLDAvis***

In [26]:
import pandas as pd
import numpy as np
import pyLDAvis

print(pd.__version__)
print(np.__version__)
print(pyLDAvis.__version__)

2.0.0
1.24.3
3.4.0


### ***0.2 Instalación de librerías***

In [27]:
import warnings
warnings.filterwarnings('ignore')

# Importación de librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unidecode
import spacy
from spacy.lang.es.examples import sentences
from unidecode import unidecode
import zipfile
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
import pyLDAvis.gensim_models
from gensim.corpora import Dictionary
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
# from transformers import AutoModel, AutoTokenizer
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from wordcloud import WordCloud
from pprint import pprint
from gensim.models import LdaModel

# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [28]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Cargar el modelo de spacy para el español
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## ***4. Recomendadores***

Para contruir el recomendador se usa un sistema de prioridad de acuerdo al sector al que pertenece el cliente, pero se hace una distinción dependiendo del sector al que pertenece el cliente como se muestra a continuación:

* ***Sector Industrial:***
       'manufactura de productos de tocador articulos de aseo',
       'combustibles lubricantes', 'comercio de repuestos',
       'edificaciones', 'gas', 'distribuidores productos consumo masivo',
       'automotores', 'grasas aceites', 'obras de infraestructura',
       'comercio de electrodomesticos', 'ferreteria', 'papel carton',
       'extraccion de carbon', 'plastico', 'maquinaria',
       'imprentas edicion', 'otros servicios financieros',
       'transporte terrestre', 'bancos', 'supermercados',
       'quimicos basicos otros quimicos', 'madera muebles',
       'servicios empresas', 'carnes frias',
       'productos metalicos', 'transporte maritimo', 'agroquimicos',
       'comisionistas de bolsa', 'cacao', 'servicios personas',
       'establecimientos educativos', 'comercio de variedades vestuario',
       'calzado marroquineria', 'confecciones',
       'acueducto alcantarillado', 'medios', 'fondos de pensiones',
       'transporte aereo', 'fiduciarias', 'molineria derivados',
       'alimentos concentrados', 'seguros', 'valor agregado',
       'extraccion de petroleo', 'textiles', 'electricidad', 'conservas',
       'ganaderia porcicultura', 'cajas de compensacion',
       'industria basica de hierro acero', 'productos lacteos',
       'pesca atun', 'aseo', 'otras actividades mineras',
       'companias de financiamiento comercial', 'flores',
       'comercio de quimicos agroquimicos', 'banano',
       'productos ceramicos', 'vidrio', 'telefonia fija',
       'bebidas no alcoholicas', 'equipo de transporte',
       'repuestos caucho', 'hoteles', 'cemento', 'bebidas alcoholicas',
       'palma africana', 'otros cultivos', 'otros manufactura',
       'resinas plasticas', 'papeles suaves', 'avicultura', 'arroz',
       'confiteria', 'azucar', 'tabaco', 'comercio de cafe',
       'pinturas barnices'

En el sector industrial la prioridad de las noticias se encuentran por:

1.   `Macroeconómicas`
2.   `Politica`
3.   `Sostenibilidad`
4.   `Innovación`
5.   `Regulaciones`
6.   `Alianzas`
7.   `Reputación`
8.   `Cultura`
9.   `Salud`
10.  `Deporte`

* Sector Salud:
       'ips','comercio de farmaceuticos', 'eps salud prepagada salud',
       'manufactura de productos farmaceuticos'

En el sector de la salud la prioridad y los pesos están función de:

1.   `Salud`
2.   `Sostenibilidad`
3.   `Innovación`
4.   `Regulaciones`
5.   `Deporte`
6.   `Cultura`
7.   `Reputación`
8.   `Macroeconómicas`
9.   `Politica`
10.  `Alianzas`


Nota: La categoría 'Otra' se indica que la eliminemos de las sugerencias ya que no son relevantes para el negocio.

### ***4.1 Recomendador: LDA - Stemming***

In [29]:
# Se carga el set de datos trabajado
df_bd_stemming_f2=pd.read_csv('../Data/DC_stemming_tema_asignado.csv')
# Se cargan los atributos de las noticias
df_noticia=pd.read_csv('../Data/noticias.csv')
# Se carga el set de datos Clientes
df_clientes=pd.read_csv('../Data/clientes.csv')


Se crea la columna con el tema de la noticia y se seleccionan las variables más relevantes para el recomendador:

In [30]:
df_noticia_stemming = df_noticia[['news_id', 'news_url_absolute','news_title', 'news_text_content', 'news_init_date']]
df_noticia_stemming = pd.merge(df_noticia_stemming, df_bd_stemming_f2[['news_id','Tema_texto']], on='news_id')
# Se retiran las noticias asignadas como 'Otra'.
df_noticia_stemming = df_noticia_stemming[df_noticia_stemming['Tema_texto'] != 'Otra']
df_noticia_stemming.head(2)

,news_id,news_url_absolute,news_title,news_text_content,news_init_date,Tema_texto
0,news10006,https://www.bluradio.com/economia/precio-dolar...,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ...",2022-07-30,Macroeconómicas
1,news10011,https://www.semana.com/economia/macroeconomia/...,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...,2022-07-30,Macroeconómicas


In [31]:
print("Valores únicos por NIT: "+str(df_noticia_stemming['news_id'].nunique()))
print("")
print("Valores nulos: ")
print(df_noticia_stemming.isnull().sum())
print("")
print("Agrupados por subsector: ")
print(df_noticia_stemming['Tema_texto'].value_counts())
print("")
print("Agrupados por fecha: ")
print(df_noticia_stemming['news_init_date'].value_counts())

Valores únicos por NIT: 20312

Valores nulos: 
news_id              0
news_url_absolute    0
news_title           0
news_text_content    0
news_init_date       0
Tema_texto           0
dtype: int64

Agrupados por subsector: 
Tema_texto
Política           4124
Innovación         3373
Alianzas           3050
Macroeconómicas    2935
Cultura            1429
Regulaciones       1260
Deportes           1180
Reputación         1138
Sostenibilidad     1059
Salud               764
Name: count, dtype: int64

Agrupados por fecha: 
news_init_date
2022-07-30    10271
2022-07-15    10041
Name: count, dtype: int64


Sólo se podrá escoger entre dos dias noticias, porque fueron las fechas de recolección.

In [32]:
# Se escribe el sector al que pertenece el cliente
condicion = lambda x: 'Salud' if x in ['ips','comercio de farmaceuticos', 'eps salud prepagada salud','manufactura de productos farmaceuticos'] else 'Industrial'
df_clientes['sector'] = df_bd_stemming_f2['subsec'].apply(condicion)
df_clientes = df_clientes[['nit', 'nombre', 'sector']]
df_clientes.head(2)

,nit,nombre,sector
0,805027024,SUPERMERCADO LA GRAN COLOMBIA SA,Industrial
1,890100026,CAMAGUEY SA,Industrial


In [33]:
print("Valores únicos por NIT: "+str(df_clientes['nit'].nunique()))
print("")
print("Valores nulos: ")
print(df_clientes.isnull().sum())
print("")
print("Agrupados por subsector: ")
print(df_clientes['sector'].value_counts())

Valores únicos por NIT: 1507

Valores nulos: 
nit       0
nombre    0
sector    0
dtype: int64

Agrupados por subsector: 
sector
Industrial    1429
Salud           78
Name: count, dtype: int64


Gran parte de la población pertenece a otros sectores diferentes al sector salud.

Ahora, se procede a crear las funciones del recomendador:

1. seleccionar_dos_filas_aleatorias: esta función seleccionará dos filas aleatorias de un grupo de noticias.
2. asignar_valor: Esta función asignará el valor que se le debe dar a la noticia de acuerdo a las definiciones del negocio y del sector del cliente.
3. recomendador: Esta función toma como insumo el nit de un cliente, el número de noticias que se le quiere mostrar y el dataframe que se le va a suministrar. La función filtra por su sector y le asigna la relevancia a la notica de acuerdo a la función `asignar_valor` que está en función del sector al que pertenece, para luego ordenarlas por temas y seleccionar 2 noticias de cada topico para presentarla al cliente, también permite visualizar el número de noticias que se desea ver (pero priorizando las de mayor relevancia para el cliente).

In [34]:
def recomendador(nit, num_noticias, fecha, dff1, dff2):
    """Se ingresan los dos df, clientes y noticias para colocar la importancia de la noticia
    según su sector y seleccionar el número de noticias y la fecha
    dff1 = Dataframe clientes
    dff2 = Dataframe de noticias
    """

    # Se les da el valor dependiendo del cliente
    def asignar_valor(row, sector):
        temas = {
            'Salud': [1, 9],
            'Sostenibilidad': [2, 3],
            'Innovación': [3, 4],
            'Regulaciones': [4, 5],
            'Deportes': [5, 10],
            'Cultura': [6, 8],
            'Reputación': [7, 7],
            'Macroeconómicas': [8, 1],
            'Política': [9, 2],
            'Alianzas': [10, 6]
        }

        tema = row['Tema_texto']

        if sector == 'Salud':
            return temas.get(tema, [11, 0])[0]
        else:
            return temas.get(tema, [11, 0])[1]

    # Se copian los df ingresado
    dff1_c = dff1.copy()
    dff2_c = dff2.copy()


    # Se identifica el sector del Nit
    s = dff1_c[dff1_c['nit'] == nit]['sector'].iloc[0]

    # Se le aplica el valor asignado con la función asignar_valor
    dff2_c = dff2_c[dff2_c['news_init_date'] == fecha]
    dff2_c['valor_asignado'] = dff2_c.apply(lambda row: asignar_valor(row, s), axis=1)

    # Ordena el DataFrame por la columna 'valor_asignado'
    dff2_c = dff2_c.sort_values(by='valor_asignado')


    # Agrupa el DataFrame por 'valor_asignado' y aplica la función para seleccionar aleatoriamente dos noticias
    resultados = dff2_c.groupby('valor_asignado', group_keys=False).apply(lambda group: group.sample(n=min(2, len(group))))

    resultados = resultados[[ 'news_init_date','Tema_texto','news_title', 'news_text_content', 'news_url_absolute']]

    resultados = resultados.rename(columns={'news_init_date': 'Fecha', 'Tema_texto': 'Tema de la Noticia','news_title': 'Titular', 'news_text_content':'Noticia', 'news_url_absolute':'Url'})


    resultados.reset_index(drop=True, inplace=True)

    return resultados.head(num_noticias)

In [35]:
# Ejemplo cliente sector Industrial
p = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
p

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,"La insolita definicion de De Mendiguren: ""La m...","El designado secretario de Produccion, Jose Ig...",https://www.forbesargentina.com/today/la-insol...
1,2022-07-30,Macroeconómicas,Bitcoin permanece en US$23.000 a medida que la...,"La criptomoneda mas grande subio hasta 4,2% a ...",https://www.larepublica.co/globoeconomia/bitco...
2,2022-07-30,Política,"'Memo Fantasma', en libertad por vencimiento d...","Guillermo Leon Acevedo, alias 'Memo Fantasma',...",https://www.laopinion.com.co/colombia/memo-fan...
3,2022-07-30,Política,"Confirmado, Alfonso Prada, nuevo ministro del ...","El presidente electo, Gustavo Petro, confirmo ...",https://confidencialcolombia.com/lo-mas-confid...
4,2022-07-30,Sostenibilidad,Los apuros de Costa Rica para sostener su band...,EL PAIS ofrece en abierto la seccion America F...,https://elpais.com/america-futura/2022-08-11/l...
5,2022-07-30,Sostenibilidad,Avianca y Viva solicitan integracion a autorid...,Las aerolineas Avianca y Viva solicitaron este...,https://gestion.pe/mundo/internacional/avianca...
6,2022-07-30,Innovación,Nueva iniciativa busca mejorar las condiciones...,Guatemala. El Foro Economico Mundial anuncio u...,https://newsinamerica.com/pdcc/economia/2022/n...
7,2022-07-30,Innovación,Actuar para transformar vidas: la apuesta de N...,La compania de salud viene consolidando un pla...,https://www.semana.com/sostenibilidad/hablan-l...
8,2022-07-30,Regulaciones,Regulacion sobre finanzas abiertas y la indust...,Direccion: Calle 70 Bis # 4 - 41 Telefonos: +5...,https://bu.com.co/es/insights/noticias/regulac...
9,2022-07-30,Regulaciones,"A 409 dias de su apresamiento, Jean Alain se e...",Despues de 409 dias de que el exprocurador Jea...,https://www.diariolibre.com/actualidad/justici...


In [36]:
# Ejemplo cliente sector Salud
q = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
q

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,Perdida de memoria por fibromialgia: como dete...,La fibromialgia es considerada una afeccion cr...,https://www.semana.com/vida-moderna/articulo/p...
1,2022-07-30,Salud,Aumentaron las muertes por COVID en Norte de S...,El quinto pico de la pandemia de la COVID-19 s...,https://www.laopinion.com.co/region/aumentaron...
2,2022-07-30,Sostenibilidad,Casi 170 kilometros de playas valencianas esta...,Casi 580.000 personas viven repartidas entre l...,https://www.epe.es/es/comunidad-valenciana/202...
3,2022-07-30,Sostenibilidad,Ecopetrol descubrio nuevo pozo de gas en en ag...,"!hola! Aqui tambien puedes encontrar ""Mis Noti...",https://www.eltiempo.com/economia/sectores/eco...
4,2022-07-30,Innovación,Integracion 5G en logistica y diseno de plataf...,NTT Data acompana en la transformacion digital...,https://www.elespanol.com/invertia/disruptores...
5,2022-07-30,Innovación,Regionalizacion y acceso al agua | Columna de ...,Colombia es una potencia hidrica mundial. Ocup...,https://www.elheraldo.co/columnas-de-opinion/j...
6,2022-07-30,Regulaciones,Aprobado el decreto que regula la creacion de ...,El Consejo de Gobierno ha aprobado el decreto ...,https://www.juntadeandalucia.es/organismos/con...
7,2022-07-30,Regulaciones,Diario HOY | Cumbre Cooperativa espera reunir ...,"La licenciada Sonia Fernandez, manifesto que e...",https://www.hoy.com.py/nacionales/cumbre-coope...
8,2022-07-30,Deportes,Seleccion Colombia Femenina Sub-20 sin miedo a...,Un inicio sonado tuvo la Seleccion Colombia Fe...,https://www.semana.com/deportes/articulo/selec...
9,2022-07-30,Deportes,Nacional de Uruguay espera contar con Luis Sua...,El Nacional uruguayo se encomendara a un Luis ...,https://gol.caracoltv.com/futbol-internacional...


El recomendador asigna de manera adecuada la priorización de noticias de acuerdo al sector.

Ahora se adecua el set de datos final, para cargarse en la base de datos de AWS.

In [37]:
df_stemming_BD = df_noticia_stemming[['news_init_date','Tema_texto','news_title','news_url_absolute']]

df_stemming_BD = df_stemming_BD.rename(columns={'news_init_date': 'Fecha',
                                  'Tema_texto': 'Tema_noticia',
                                  'news_title': 'Titulo_noticia',
                                  'news_url_absolute' : 'Url'})

df_stemming_BD.head(3)

,Fecha,Tema_noticia,Titulo_noticia,Url
0,2022-07-30,Macroeconómicas,Precio dolar hoy: la cotizacion de la divisa a...,https://www.bluradio.com/economia/precio-dolar...
1,2022-07-30,Macroeconómicas,Es cierto: El presidente Gustavo Petro quiere ...,https://www.semana.com/economia/macroeconomia/...
2,2022-07-30,Política,Gustavo Petro: ?Que dice el informe final de s...,https://elcomercio.pe/respuestas/que/gustavo-p...


In [38]:
df_clientes_BD = df_clientes.rename(columns={'nit' : 'Nit', 'nombre':'Nombre', 'sector':'Sector'})
df_clientes_BD.head(3)

,Nit,Nombre,Sector
0,805027024,SUPERMERCADO LA GRAN COLOMBIA SA,Industrial
1,890100026,CAMAGUEY SA,Industrial
2,801004045,DON POLLO SAS,Industrial


In [39]:
ruta_csv = '../Data/df_stemming_BD.csv'
df_stemming_BD.to_csv(ruta_csv, index=False)

ruta_csv = '../Data/df_clientes_BD.csv'
df_clientes_BD.to_csv(ruta_csv, index=False)

### ***4.2 Recomendador: LDA - Lematización***

In [40]:
# Se carga el set de datos trabajado
df_bd_lemat_f2=pd.read_csv('../Data/DC_lematizado_tema_asignado.csv')

Se crea la columna con el sector y se seleccionan las variables más relevantes para el recomendador:

In [41]:
df_noticia_lemat = df_noticia[['news_id', 'news_url_absolute','news_title', 'news_text_content', 'news_init_date']]
df_noticia_lemat = pd.merge(df_noticia_lemat, df_bd_lemat_f2[['news_id','Tema_texto']], on='news_id')
# Se retiran las noticias asignadas como 'Otra'.
df_noticia_lemat = df_noticia_lemat[df_noticia_lemat['Tema_texto'] != 'Otra']
df_noticia_lemat.head(2)

,news_id,news_url_absolute,news_title,news_text_content,news_init_date,Tema_texto
0,news10006,https://www.bluradio.com/economia/precio-dolar...,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ...",2022-07-30,Macroeconómicas
1,news10011,https://www.semana.com/economia/macroeconomia/...,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...,2022-07-30,Macroeconómicas


In [42]:
print("Valores únicos por NIT: "+str(df_noticia_lemat['news_id'].nunique()))
print("")
print("Valores nulos: ")
print(df_noticia_lemat.isnull().sum())
print("")
print("Agrupados por subsector: ")
print(df_noticia_lemat['Tema_texto'].value_counts())
print("")
print("Agrupados por fecha: ")
print(df_noticia_lemat['news_init_date'].value_counts())

Valores únicos por NIT: 19813

Valores nulos: 
news_id              0
news_url_absolute    0
news_title           0
news_text_content    0
news_init_date       0
Tema_texto           0
dtype: int64

Agrupados por subsector: 
Tema_texto
Política           3705
Macroeconómicas    3358
Alianzas           3350
Regulaciones       3232
Cultura            1481
Innovación         1465
Reputación         1317
Salud               717
Deportes            651
Sostenibilidad      537
Name: count, dtype: int64

Agrupados por fecha: 
news_init_date
2022-07-30    10083
2022-07-15     9730
Name: count, dtype: int64


Ahora, se procede a consumir las funciones creadas cuando se creó el recomendador por stemming:

In [43]:
# Ejemplo cliente sector Industrial
r = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
r

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,"Caen ofertas laborales en Estados Unidos, una ...",El numero de puestos vacantes en Estados Unido...,https://www.semana.com/economia/macroeconomia/...
1,2022-07-30,Macroeconómicas,Reforma tributaria: este seria el impacto en e...,El periodo presidencial de Gustavo Petro comen...,https://consultorsalud.com/reforma-tributaria-...
2,2022-07-30,Política,Calvino nombra a Elena Manzanera nueva preside...,Elena Manzanera Diaz sera la nueva presidenta ...,https://www.elespanol.com/invertia/economia/ma...
3,2022-07-30,Política,"""Estamos listos para pasar mercancias a Venezu...","Este viernes, la agenda del Congreso Empresari...",https://noticias.caracoltv.com/economia/estamo...
4,2022-07-30,Sostenibilidad,"Ecopetrol hizo nuevo descubrimiento de gas, es...",Desde Ecopetrol se dio a conocer que el pozo e...,https://www.wradio.com.co/2022/08/10/ecopetrol...
5,2022-07-30,Sostenibilidad,La Amazonia arde,La Amazonia arde Con la intencion de mostrarle...,https://www.radionacional.co/podcast/mujeres-d...
6,2022-07-30,Innovación,Crece la influencia de los colaboradores: 47% ...,El capital humano en las empresas de todo el m...,https://eldinero.com.do/205029/crece-la-influe...
7,2022-07-30,Innovación,Audi Progressive Retail es un nuevo concepto i...,' Compartiendo este articulo usted puede ayuda...,https://valenciacars.blogspot.com/2022/08/audi...
8,2022-07-30,Regulaciones,Colombia Mayor: anuncian pagos de julio y apli...,Bogota. D.C. Agosto 1 de 2022 . Colombia Mayor...,https://prosperidadsocial.gov.co/Noticias/colo...
9,2022-07-30,Regulaciones,DiDi ve potencial para ampliar sus soluciones ...,La pandemia trunco o freno varios planes de em...,https://www.forbesargentina.com/liderazgo/didi...


In [44]:
# Ejemplo cliente sector Salud
q = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
q

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,Todo a punto para celebrar la Noche Viva Fueng...,Utilizamos cookies propias y de terceros para ...,https://www.benalgo.es/noticias/todo-a-punto-p...
1,2022-07-30,Salud,El Sescam refuerza la formacion en Psiquiatria...,"El Hospital Universitario de Ciudad Real, refe...",https://diariosanitario.com/salud-mental-ciuda...
2,2022-07-30,Sostenibilidad,La 'dieta de los romanos' que reivindican los ...,Un proyecto de la Institucion Mila y Fontanals...,https://www.elespanol.com/ciencia/nutricion/20...
3,2022-07-30,Sostenibilidad,Gobierno Duque desistio de adjudicar minas de ...,"El ministro de Minas y Energia, Diego Mesa, de...",https://www.semana.com/economia/macroeconomia/...
4,2022-07-30,Innovación,Global Reductores de engranajes cicloidales Me...,El informe de investigacion de mercado global ...,https://www.loscronistas.mx/global-reductores-...
5,2022-07-30,Innovación,6 hinchables para practicar deporte en la pisc...,No hay nada como pasarlo en grande entre chapu...,https://www.telecinco.es/regalos/ofertas-hogar...
6,2022-07-30,Regulaciones,Superservicios devolvio Eicviro E.S.P. al muni...,La Superintendencia de Servicios Publicos Domi...,https://www.laopinion.com.co/region/superservi...
7,2022-07-30,Regulaciones,La Bolsa Mercantil de Colombia: un actor relev...,"miercoles, 10 de agosto de 2022 La Bolsa Merca...",https://www.asuntoslegales.com.co/analisis/ser...
8,2022-07-30,Deportes,Virgil van Dijk sera embajador de Tundra Esports,"""Estoy extremadamente emocionado de unirme a T...",https://442.perfil.com/noticias/esports/virgil...
9,2022-07-30,Deportes,Hoy comenzo el Mundial Ecuestre 2022 que conta...,Hoy comenzo el Mundial Ecuestre 2022 de la Fed...,https://www.agronegocios.co/ferias/hoy-comenzo...


El recomendador asigna de manera adecuada la priorización de noticias de acuerdo al sector.


Ahora se adecua el set de datos final, para cargarse en la base de datos de AWS.

In [45]:
df_lemat_BD = df_noticia_lemat[['news_init_date','Tema_texto','news_title','news_url_absolute']]

df_lemat_BD = df_lemat_BD.rename(columns={'news_init_date': 'Fecha',
                                  'Tema_texto': 'Tema_noticia',
                                  'news_title': 'Titulo_noticia',
                                  'news_url_absolute' : 'Url'})

df_lemat_BD.head(3)

,Fecha,Tema_noticia,Titulo_noticia,Url
0,2022-07-30,Macroeconómicas,Precio dolar hoy: la cotizacion de la divisa a...,https://www.bluradio.com/economia/precio-dolar...
1,2022-07-30,Macroeconómicas,Es cierto: El presidente Gustavo Petro quiere ...,https://www.semana.com/economia/macroeconomia/...
2,2022-07-30,Política,Gustavo Petro: ?Que dice el informe final de s...,https://elcomercio.pe/respuestas/que/gustavo-p...


In [46]:
ruta_csv = '../Data/df_lemat_BD.csv'
df_lemat_BD.to_csv(ruta_csv, index=False)

## ***5. Resultados***

En este apartado se visualizan dos resultados de las sugerencias de las noticias para los clientes, uno del sector salud y otro del sector industrial. La idea es mostrar 10 noticias para cada uno de los modelos y revisar la asertividad de las noticias.

**Sector salud - LDA (Stemming)**

In [47]:
# Ejemplo cliente sector Salud
q = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
q

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,MinHacienda subasto TES en pesos por $975.000 ...,Copyright (c) 2022 Publicaciones Semana S.A NI...,https://www.semana.com/economia/capsulas/artic...
1,2022-07-30,Salud,Aniversario 30 de BioCen: una apuesta de la ci...,Por: Sheila Noda Alonso El Centro Nacional de ...,https://www.radiohc.cu/especiales/exclusivas/2...
2,2022-07-30,Sostenibilidad,Reserva ecologica cubana impulsa manejo sosten...,"La Habana, 1 ago.- La Reserva Ecologica Hatibo...",https://www.ipscuba.net/medio-ambiente/reserva...
3,2022-07-30,Sostenibilidad,Ecopetrol comenzo a importar gas natural para ...,"Ecopetrol y Cenit, filial de transporte del Gr...",https://www.semana.com/economia/macroeconomia/...
4,2022-07-30,Innovación,Chile implementa medidas contra el robo de veh...,Mas de mil millones de pesos comprometidos por...,https://webpicking.com/chile-implementa-medida...
5,2022-07-30,Innovación,Loteria de Boyaca y de Cundinamarca firman ali...,El proximo 2 de septiembre se hara el sorteo ...,https://www.boyaca.gov.co/loteria-de-boyaca-y-...
6,2022-07-30,Regulaciones,El principio de plurinacionalidad. - Diario Co...,El objetivo de esta nota es discutir acerca de...,https://www.diarioconstitucional.cl/2022/07/30...
7,2022-07-30,Regulaciones,Lanzan propuestas sindicatos y asociaciones pa...,Representantes de diversos sindicatos y asocia...,https://www.eloccidental.com.mx/local/lanzan-p...
8,2022-07-30,Deportes,Alianza Petrolera no pudo en casa con At. Naci...,En la fecha 6 del torneo Colombia - Liga Betp...,https://www.tycsports.com/colombia/futbol-de-c...
9,2022-07-30,Deportes,Argentina y Brasil se citan en la final del Su...,Este torneo clasifico a los cuatro semifinalis...,https://www.abc.com.py/deportes/basquetbol/202...


**Sector salud - LDA (Lematización)**

In [48]:
# Ejemplo cliente sector Salud
t = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
t

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,"Manana, calor en casi todo el pais y fuertes t...","El cantante Rayden , rompiendo tabues sobre la...",https://www.efe.com/efe/espana/sociedad/manana...
1,2022-07-30,Salud,UNICEF: La lactancia materna sigue siendo un d...,"Como cada ano del 1 al 7 de agosto , se celebr...",https://www.diariolibre.com/actualidad/naciona...
2,2022-07-30,Sostenibilidad,Grupo Ecopetrol inicio importacion de gas licu...,Ecopetrol y Cenit pusieron en marcha un proyec...,https://www.valoraanalitik.com/2022/08/02/grup...
3,2022-07-30,Sostenibilidad,"La relevancia de la genetica, la energia y las...","Para la agricultura, el nexo entre el agua y l...",https://www.portalfruticola.com/noticias/2022/...
4,2022-07-30,Innovación,?Que son los NFT y como funcionan en el metave...,"Santiago, 10 de agosto de 2022. El mundo esta ...",https://www.trendtic.cl/2022/08/que-son-los-nf...
5,2022-07-30,Innovación,Multimax crea alianza con empresa Camarounds p...,(Ciudad de Panama-ANPanama) La empresa Multima...,https://www.anpanama.com/12466-Multimax-crea-a...
6,2022-07-30,Regulaciones,Retractacion a tiempo,El 9 FAM consigna la retractacion a tiempo com...,https://listindiario.com/puntos-de-vista/2022/...
7,2022-07-30,Regulaciones,Castagneto prometio hacer una AFIP que facilit...,El nuevo titular de la Administracion Federal...,https://www.cadena3.com/noticia/politica-y-eco...
8,2022-07-30,Deportes,Se reivindico. Nicolas Benedetti marco nuevo g...,"Como a bordo de un carrusel, parece ir el juga...",https://www.infobae.com/america/colombia/2022/...
9,2022-07-30,Deportes,Copa Sudamericana de futsal: semifinal colombi...,Los colombianos Caciques del Quindio y Faraone...,https://www.elquindiano.com/noticia/36059/copa...


**Sector Industrial - LDA (Stemming)**

In [49]:
# Ejemplo cliente sector Industrial
p = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
p

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,El estado de regular a malo en el trigo alcanz...,Luego de que el Banco Central (BCRA) prohibie...,https://rufinoweb.com.ar/el-estado-de-regular-...
1,2022-07-30,Macroeconómicas,"Las reservas, el dolar y la informacion: la ne...","Desde hace algunos anos, los gobiernos argenti...",https://www.forbesargentina.com/macroeconomia/...
2,2022-07-30,Política,Canal del Dique: las 4 dudas que frenaron la m...,En la Agencia Nacional de Infraestructura (ANI...,https://www.eltiempo.com/unidad-investigativa/...
3,2022-07-30,Política,"Operacion Artemisa: Susana Muhamad, minAmbient...","Susana Muhamad, la ministra de Ambiente design...",https://www.elespectador.com/ambiente/operacio...
4,2022-07-30,Sostenibilidad,MINI Cooper S Countryman ALL4 explora los Hamp...,',https://valenciacars.blogspot.com/2022/08/mini...
5,2022-07-30,Sostenibilidad,Australia estudia frenar las exportaciones de ...,"La medida del Gobierno, tras una recomendacion...",https://www.worldenergytrade.com/politica/ocea...
6,2022-07-30,Innovación,Vizzotti y Filmus analizaron los avances en el...,"La ministra de Salud de la Nacion, Carla Vizzo...",https://www.lacapitalmdp.com/vizzotti-y-filmus...
7,2022-07-30,Innovación,"Ingreso Solidario hoy | Quien lo cobra, ultima...",Continuan los ciclos de pago del programa crea...,https://colombia.as.com/actualidad/ingreso-sol...
8,2022-07-30,Regulaciones,"""Fortalecer la representacion politica de la s...","""El objetivo es fortalecer la representacion p...",https://www.diarioeltiempo.com.ar/nota--fortal...
9,2022-07-30,Regulaciones,HONOR da 5 consejos para hacer fotos como prof...,Las pensiones en Bolivia se encuentran en una ...,https://www.la-razon.com/financiero/2022/07/31...


**Sector Industrial - LDA (Lematización)**

In [50]:
# Ejemplo cliente sector Industrial
r = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
r

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,La digitalizacion impulsa la competencia en el...,La expansion del credito de consumo se acelero...,https://larepublica.pe/economia/2022/08/08/bcr...
1,2022-07-30,Macroeconómicas,?Como puede influir la reapertura de frontera ...,"En el trimestre abril-junio, Cucuta se ubico c...",https://www.eluniversal.com.co/colombia/como-p...
2,2022-07-30,Política,?Que esperar de la relacion entre Ecuador y Co...,QUITO -- Pragmatismo y pensamiento tecnico ser...,https://www.bloomberglinea.com/2022/08/09/que-...
3,2022-07-30,Política,Un nuevo escenario de aprendizaje llega a Colo...,La primera Catedra Unesco para Iberoamerica en...,https://www.eltiempo.com/contenido-comercial/u...
4,2022-07-30,Sostenibilidad,Algas devastadoras asfixian las costas del Caribe,Una cantidad record de algas asfixia las costa...,https://www.diariolibre.com/planeta/medioambie...
5,2022-07-30,Sostenibilidad,Llamado urgente por el bienestar del planeta,"Durante las ultimas semanas, la poblacion mund...",https://www.eltiempo.com/mas-contenido/llamado...
6,2022-07-30,Innovación,"Krealo de Credicorp: ""Ojala podamos hacer dos ...",--?Cuales son los lineamientos de Krealo para ...,https://elcomercio.pe/economia/dia-1/krealo-de...
7,2022-07-30,Innovación,"Llego al pais IYO Movil, una opcion de telefon...",La plataforma Suma Movil de servicios moviles ...,https://www.larepublica.co/empresas/llega-al-p...
8,2022-07-30,Regulaciones,Macri recargado: cruzo a gremialistas K y aseg...,En medio de la interna que atraviesa Juntos po...,https://www.perfil.com/noticias/politica/mauri...
9,2022-07-30,Regulaciones,Tren Bala llega a cines con Bad Bunny y Brad Pitt,En la conversacion tambien esta Brian Tyree He...,https://www.elcolombiano.com/cultura/cine/bad-...
